# Capítulo 5

Em primeiro lugar, vamos importar os pacotes básicos para o código que temos a seguir.

In [18]:
import numpy as np
import matplotlib.pyplot as plt
import scipy
import pandas as pd 
import math
import random
import statsmodels.api as sm
import statsmodels.formula.api as smf
from statsmodels.graphics.regressionplots import *
from sklearn import datasets, linear_model
from sklearn.model_selection import KFold, LeaveOneOut, cross_val_score
from sklearn.preprocessing import PolynomialFeatures
from sklearn.linear_model import LinearRegression
from sklearn.pipeline import Pipeline
from collections import OrderedDict

## 5.3.1 Conjunto de validação

Usaremos os dados `Auto`, descartando as observação com entradas NAs.
.

In [19]:
Auto = pd.read_csv('../data/Auto.csv', header=0, na_values='?')
Auto = Auto.dropna().reset_index(drop=True)

In [20]:
print(Auto.shape)

(392, 9)


In [21]:
print(Auto.head())

    mpg  cylinders  displacement  horsepower  weight  acceleration  year  \
0  18.0          8         307.0       130.0    3504          12.0    70   
1  15.0          8         350.0       165.0    3693          11.5    70   
2  18.0          8         318.0       150.0    3436          11.0    70   
3  16.0          8         304.0       150.0    3433          12.0    70   
4  17.0          8         302.0       140.0    3449          10.5    70   

   origin                       name  
0       1  chevrolet chevelle malibu  
1       1          buick skylark 320  
2       1         plymouth satellite  
3       1              amc rebel sst  
4       1                ford torino  


Como lidaremos com splits aleatórios, precisamos fixar um seed.

In [22]:
np.random.seed(1)

Agora, podemos gerar o conjunto de treino e teste escolhendo metade da amostra.

In [23]:
train_idx = np.random.choice(Auto.shape[0], 196, replace=False)
in_train = np.in1d(range(Auto.shape[0]), train_idx)

In [24]:
in_train

array([ True, False, False, False,  True,  True,  True, False,  True,
        True, False,  True,  True,  True,  True, False,  True,  True,
        True,  True, False, False, False,  True, False, False, False,
        True,  True,  True, False, False, False, False,  True, False,
       False, False,  True,  True, False,  True,  True, False,  True,
       False, False, False, False, False, False,  True, False, False,
       False, False, False, False,  True,  True, False,  True,  True,
       False, False,  True, False,  True, False, False,  True, False,
       False,  True, False, False, False, False,  True,  True,  True,
        True,  True, False,  True,  True, False, False,  True,  True,
        True,  True,  True,  True, False,  True, False, False,  True,
        True,  True, False,  True, False, False,  True,  True,  True,
        True, False,  True,  True,  True, False, False, False, False,
        True, False,  True,  True, False,  True,  True,  True,  True,
       False,  True,

Vamos selecionar os dados de treino `Auto[in_train]` para treinar uma regressão linear de `mpg` em `horsepower`.

In [25]:
lm = smf.ols('mpg~horsepower', data = Auto[in_train]).fit()
print(lm.summary())

                            OLS Regression Results                            
Dep. Variable:                    mpg   R-squared:                       0.620
Model:                            OLS   Adj. R-squared:                  0.618
Method:                 Least Squares   F-statistic:                     316.4
Date:                Thu, 11 Jan 2024   Prob (F-statistic):           1.28e-42
Time:                        17:50:40   Log-Likelihood:                -592.07
No. Observations:                 196   AIC:                             1188.
Df Residuals:                     194   BIC:                             1195.
Df Model:                           1                                         
Covariance Type:            nonrobust                                         
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
Intercept     40.3338      1.023     39.416      0.0

Vamos ver como o modelo se comporta no treino e no teste em termos de erro médio quadrático (MSE).

In [26]:
preds_train = lm.predict(Auto[in_train])
MSE = np.mean((Auto['mpg'][in_train] - preds_train)**2)
print(f"Train error for 1st order model: {MSE}")

Train error for 1st order model: 24.62301015144335


In [27]:
in_test = ~in_train
preds_test = lm.predict(Auto[in_test])
MSE = np.mean((Auto['mpg'][in_test] - preds_test)**2)
print(f"Test error for 1st order model: {MSE}")

Test error for 1st order model: 23.36190289258724


Curiosamente, parece que nosso erro de teste é melhor do que de treino. Isso em geral é sinal de que poderíamos usar um modelo mais flexível para os dados. Uma possibilidade é incluir mais previsores. Vamos olhar para o erro de teste incluindo um modelo quadrático e um cúbico.

In [28]:
lm2 = smf.ols('mpg~horsepower + I(horsepower ** 2.0)', data = Auto[in_train]).fit()
preds_test = lm2.predict(Auto[in_test])
MSE = np.mean((Auto['mpg'][in_test] - preds_test)**2)
print(f"Test error for 2nd order model: {MSE}")

Test error for 2nd order model: 20.252690858350157


In [29]:
lm3 = smf.ols('mpg~horsepower + I(horsepower ** 2.0) + I(horsepower ** 3.0)', data = Auto[in_train]).fit()
preds_test = lm3.predict(Auto[in_test])
MSE = np.mean((Auto['mpg'][in_test] - preds_test)**2)
print(f"Test error for 3rd order model: {MSE}")

Test error for 3rd order model: 20.325609365598602


Como assinalado no livro, um modelo quadrático parece razoável, e um modelo cúbico parece mais flexível do que o necessário. Uma outra evidência para isso é o p-valor do termo cúbico, que não é estatisticamente signficativo.

In [30]:
print(lm3.summary())

                            OLS Regression Results                            
Dep. Variable:                    mpg   R-squared:                       0.722
Model:                            OLS   Adj. R-squared:                  0.717
Method:                 Least Squares   F-statistic:                     165.9
Date:                Thu, 11 Jan 2024   Prob (F-statistic):           4.60e-53
Time:                        17:50:46   Log-Likelihood:                -561.56
No. Observations:                 196   AIC:                             1131.
Df Residuals:                     192   BIC:                             1144.
Df Model:                           3                                         
Covariance Type:            nonrobust                                         
                           coef    std err          t      P>|t|      [0.025      0.975]
----------------------------------------------------------------------------------------
Intercept               66.5200 

## 5.3.2 Leave-One-Out Cross-Validation

A estimativa de erro via LOOCV é feita usando cada ponto dos dados como o conjunto de validação (com o resto dos dados para treino) e depois tirando a média. Para rodar LOOCV, é conveniente usar scikit-learn. Vamos primeiro nos certificar de que os resultados usando o `statsmodels` e `sklearn` são iguais.

In [31]:
ols_fit = smf.ols('mpg~horsepower', data = Auto).fit()
print(ols_fit.params)

Intercept     39.935861
horsepower    -0.157845
dtype: float64


In [32]:
# vamos retreinar o modelo no sklearn
X = pd.DataFrame(Auto.horsepower)
y = Auto.mpg
model = LinearRegression()
model.fit(X, y)

print(model.intercept_)
print(model.coef_)

39.935861021170474
[-0.15784473]


Para usar LOOCV, podemos ou usar a função `LeaveOneOut()` ou, notando que LOOCV é equivalente a CV com $k=n$, usar a função `KFold()`.

In [37]:
# k_fold = LeaveOneOut()
cv = KFold(n_splits=X.shape[0]) 
cv_test_errors = cross_val_score(model, X, y, cv=cv, scoring='neg_mean_squared_error', n_jobs=-1)
print(np.mean(-cv_test_errors))

24.231513517929226


Podemos repetir esse procedimento para regressões polinomiais cada vez mais complexas e investigar o erro estimado por LOOCV.

In [38]:
for poly_order in range(1, 21, 2):
    model = Pipeline([('polynomial_regression', PolynomialFeatures(degree=poly_order)), ('linear', LinearRegression())])
    cv_test_errors = cross_val_score(model, X, y, cv=cv,  scoring = 'neg_mean_squared_error', n_jobs=-1)
    print(f"Order {poly_order}: {np.mean(-cv_test_errors)}")

Order 1: 24.231513517929226
Order 3: 19.334984064089728
Order 5: 19.033209457446823
Order 7: 19.126045435248674
Order 9: 19.13385027310686
Order 11: 19.11730751792413
Order 13: 27.763425455307395
Order 15: 35.29332499720309
Order 17: 43.65448563477038
Order 19: 60.96389586311333


Aparentemente deveríamos escolher o modelo de ordem 3, pois é o que tem menor erro de LOOCV.

## 5.3.3 k-Fold Cross-Validation

Para rodar $k$-fold CV, basta usar escolher o número de folds.

In [39]:
np.random.seed(1)
k = 10

for poly_order in range(1, 21, 2):
    model = Pipeline([('polynomial_regression', PolynomialFeatures(degree=poly_order)), ('linear', LinearRegression())])
    k_fold = KFold(n_splits=k) 
    cv_test_errors = cross_val_score(model, X, y, cv=k_fold,  scoring = 'neg_mean_squared_error', n_jobs = -1)
    print(f"Order {poly_order}: {np.mean(-cv_test_errors)}")

Order 1: 27.439933652339857
Order 3: 21.336606183429943
Order 5: 20.905671818048763
Order 7: 20.952932899234856
Order 9: 21.036784344385175
Order 11: 21.42397996624395
Order 13: 30.810090119753227
Order 15: 39.53672473255026
Order 17: 48.27499212444177
Order 19: 64.02372757596336


## 5.3.4 Bootstrap

Bootstrap significa estimar incerteza nos dados via reamostragem com reposição. Usamos amostras dos dados originais preservando o mesmo tamanho total dos dados para evitar que efeitos de tamanho da amostra influenciem nas estimativas. Essa técnica é muito útil para desenvolver intervalos de confiança e testes de hipótese.

Nesta subseção, vamos usar os dados `Portfolio`.

In [40]:
Portfolio = pd.read_csv('../data/Portfolio.csv', header=0)

Para usar o bootstrap, primeiro criamos uma função que recebe uma replicação dos dados e retorna a quantidade de interesse. Seguindo o exemplo do texto, vamos escolher a função $\alpha$, definida em (5.7).

In [41]:
def alpha_fn(data, index):
    X = data.X.iloc[index]
    Y = data.Y.iloc[index]
    return (np.var(Y) - np.cov(X,Y)[0,1])/(np.var(X) + np.var(Y) - 2 * np.cov(X, Y)[0,1])

Por exemplo, suponha que queiramos o valor de `alpha_fn()` calculada nas primeiras 100 instâncias dos dados:

In [42]:
alpha_fn(Portfolio, range(0,100))

0.5766511516104116

Agora, precisamos gerar uma amostra com reposição de um conjunto de índices, satisfazendo um certo tamanho total. Vamos usar algumas funções de Numpy para isso, e depois instanciar a função `alpha_fn()`.

In [43]:
np.sort(np.random.choice(range(0, 100), size=100, replace=True))

array([ 0,  1,  1,  3,  3,  3,  5,  6,  7,  7,  8,  8,  9,  9,  9, 10, 11,
       12, 13, 13, 14, 15, 15, 16, 18, 20, 21, 22, 22, 23, 23, 23, 24, 25,
       25, 25, 25, 26, 26, 27, 28, 29, 30, 32, 32, 34, 37, 37, 38, 41, 43,
       46, 47, 49, 50, 50, 52, 55, 57, 57, 57, 60, 61, 62, 63, 64, 64, 65,
       67, 68, 68, 68, 70, 71, 71, 71, 72, 72, 74, 75, 76, 76, 77, 79, 80,
       81, 82, 83, 84, 86, 87, 87, 87, 87, 88, 88, 92, 94, 96, 98])

In [44]:
# recall the previous function with a random set of input. 
alpha_fn(Portfolio, np.random.choice(range(0, 100), size=100, replace=True))

0.46805758220825666

Note que o resultado é diferente da função calculada nos 100 primeiros índices dos dados, pois o valor acima usa uma amostra diferente. O importante é que, tomando várias amostras de 100 indíces escolhidas com reposição, ganhamos uma estimativa válida da variabilidade de `alpha_fn()`.

In [45]:
def boot_python(data, input_function, replications):
    n = Portfolio.shape[0]
    sample_replications = np.random.randint(0, n, (replications, n))  # draw (replications) samples with numbers from 0 to n
    bootstrap_replications = np.zeros(replications)
    for i in range(len(sample_replications)):
        bootstrap_replications[i] = input_function(data, sample_replications[i])
    
    return {'Mean': np.mean(bootstrap_replications), 'std. dev': np.std(bootstrap_replications)}

In [46]:
boot_python(Portfolio, alpha_fn, 1000)

{'Mean': 0.5797747629892958, 'std. dev': 0.09393470598305134}

Ou seja, em média a função `alpha_fn` nos dados é 0.58, com um desvio padrão de 0.1.